# Using for iterator in Output Models

When generating structured outputs their may be the case where need a list of structured outputs based on the input given. The most common solution for this case is to run a task prompt multiple times for each input item. However, this can be inefficient and time consuming. To solve this problem, we can use the for iterator in the output model to generate a list of structured outputs based on the input given.


## Preparing the Input List

We will use our previous InputObject (Book) and generate a list of sequels by passing an BaseModel with "sequels: list[str]" as attribute.

In [1]:
from typing import Literal
from pydantic import BaseModel
from llmp.services.program import Program


class BookInput(BaseModel):
    book_title: str
    book_author: str
    release_year: int
    
class SequalOutput(BaseModel):
    sequels: list[str]
    

program = Program("gen sequels", BookInput, SequalOutput, debug=True)

Running instruction generation from scratch.
=== Format Inputs (Pre) ===
Prompt: I have a task that takes an input and produces an output. Below you will find the input and output models for that task which define and restrict the possible input objects and the desired output object.
I have lost the instruction for that task. Can you help me work out what it was?
Think step-by-step to reverse engineer the task instruction. What other inputs might look like with the mentioned keys? What is the relationship between them? What might be the right operation (label, extract, compress, generate, etc.) to get from any input object with that structure to the desired output object?
Please return a response in yaml format using the following schema:
```yaml
Reasoning: <str>
Instruction: <str>

```
Do not include any other information or explanation to your response, 
so that your response can be parsed with yaml.safe_load().
Remember to set the value in quotes using the Double quotation marks 
wh

In [2]:
input_data={
    "book_title": "Harry Potter",
    "book_author": "J. K. Rowling",
    "release_year": 1997
}

output = program(input_data)

print(output.sequels)

['Harry Potter and the Chamber of Secrets', 'Harry Potter and the Prisoner of Azkaban', 'Harry Potter and the Goblet of Fire', 'Harry Potter and the Order of the Phoenix', 'Harry Potter and the Half-Blood Prince', 'Harry Potter and the Deathly Hallows']


## Iterating over the Input List

In [3]:
from pydantic import Field


class InputObject(BaseModel):
    book_list: list[str]
    
    
class OutputObject(BaseModel):
    book_data: list[BookInput] = Field(rule="for each $book in {book_list}")
    
    
program_iter = Program("gen sequels data", InputObject, OutputObject)   

In [4]:
result = program_iter({"book_list": output.sequels})

print(result.book_data)

[{'book_title': 'Harry Potter and the Chamber of Secrets', 'book_author': 'J.K. Rowling', 'release_year': 1998}, {'book_title': 'Harry Potter and the Prisoner of Azkaban', 'book_author': 'J.K. Rowling', 'release_year': 1999}, {'book_title': 'Harry Potter and the Goblet of Fire', 'book_author': 'J.K. Rowling', 'release_year': 2000}, {'book_title': 'Harry Potter and the Order of the Phoenix', 'book_author': 'J.K. Rowling', 'release_year': 2003}, {'book_title': 'Harry Potter and the Half-Blood Prince', 'book_author': 'J.K. Rowling', 'release_year': 2005}, {'book_title': 'Harry Potter and the Deathly Hallows', 'book_author': 'J.K. Rowling', 'release_year': 2007}]


In [5]:
program_iter.job.instruction

"Given a list of book titles, create a list of dictionaries where each dictionary represents a book and contains the book's title, author, and release year."

In [8]:
from llmp.data_model.job_record import load_engine_from_job

engine = load_engine_from_job(program.job)

template = "{format_instruction}"

print(engine.prompt_builder.build(book_list=output.sequels))

Given a list of book titles, create a list of dictionaries where each dictionary represents a book and contains the book title, author, and release year.
# Format instructions

Please return a response in yaml format using the following schema:
```yaml
Book data:
- book_title: <str>
  book_author: <str>
  release_year: <int>
- book_title: <str>
  book_author: <str>
  release_year: <int>
- book_title: <str>
  book_author: <str>
  release_year: <int>
- book_title: <str>
  book_author: <str>
  release_year: <int>
- book_title: <str>
  book_author: <str>
  release_year: <int>
- book_title: <str>
  book_author: <str>
  release_year: <int>

```
Do not include any other information or explanation to your response, 
so that your response can be parsed with yaml.safe_load().
Remember to set the value in quotes using the Double quotation marks 
when values are multiline strings or contain ':'.




{input}


In [4]:
result = program({"book_list": output.sequels})

print(result.book_data)

["Validation failed with errors:\nKeys ['book_title', 'book_author'] not in output\nKeys ['book_title', 'book_author'] not in output\nKeys ['book_title', 'book_author'] not in output\nKeys ['book_title', 'book_author'] not in output\nKeys ['book_title', 'book_author'] not in output\nKeys ['book_title', 'book_author'] not in output\nFor output: Book data:\n- title: Harry Potter and the Chamber of Secrets\n  author: J.K. Rowling\n  release_year: 1998\n- title: Harry Potter and the Prisoner of Azkaban\n  author: J.K. Rowling\n  release_year: 1999\n- title: Harry Potter and the Goblet of Fire\n  author: J.K. Rowling\n  release_year: 2000\n- title: Harry Potter and the Order of the Phoenix\n  author: J.K. Rowling\n  release_year: 2003\n- title: Harry Potter and the Half-Blood Prince\n  author: J.K. Rowling\n  release_year: 2005\n- title: Harry Potter and the Deathly Hallows\n  author: J.K. Rowling\n  release_year: 2007\n\n\nParsed Output:\n{'book_data': [{'title': 'Harry Potter and the Cham

ValidationError: Validation failed with errors:
Keys ['book_title', 'book_author'] not in output
Keys ['book_title', 'book_author'] not in output
Keys ['book_title', 'book_author'] not in output
Keys ['book_title', 'book_author'] not in output
Keys ['book_title', 'book_author'] not in output
Keys ['book_title', 'book_author'] not in output
For output: Book data:
- title: Harry Potter and the Chamber of Secrets
  author: J.K. Rowling
  release_year: 1998
- title: Harry Potter and the Prisoner of Azkaban
  author: J.K. Rowling
  release_year: 1999
- title: Harry Potter and the Goblet of Fire
  author: J.K. Rowling
  release_year: 2000
- title: Harry Potter and the Order of the Phoenix
  author: J.K. Rowling
  release_year: 2003
- title: Harry Potter and the Half-Blood Prince
  author: J.K. Rowling
  release_year: 2005
- title: Harry Potter and the Deathly Hallows
  author: J.K. Rowling
  release_year: 2007


Parsed Output:
{'book_data': [{'title': 'Harry Potter and the Chamber of Secrets

NameError: name 'result' is not defined